In [1]:
import pandas as pd
import os

In [2]:
Desktop = os.path.join(os.path.expanduser("~"), "Desktop")

filepath_root = os.path.join(Desktop, "LBP_Revision")

os.path.exists(filepath_root)

path_inclusion_crit_other = os.path.join(filepath_root,'inclusion_codes_other.xlsx')

path_inclusion_crit_scoliosis = os.path.join(filepath_root,'inclusion_codes_scoliosis.xlsx')


In [3]:
def get_cpts_fromfile(filepath=''):
    df_cpt = pd.read_excel(filepath, dtype=str, sheet_name='CPT Codes')
    return df_cpt['CPT Code']       

def get_icds_fromfile(filepath=''):        
    df_icd = pd.read_excel(filepath,dtype=str, sheet_name='ICD Codes')
    return df_icd['ICD Code']

In [4]:
LBP_other_filename = 'LBP_other_patients'
LBP_scoliosis_filename = 'LBP_scoliosis_patients'
LBP_other_all_data = pd.read_csv(filepath_root + '/' + LBP_other_filename + '.csv', encoding = 'utf8',dtype=str)
LBP_scoliosis_all_data = pd.read_csv(filepath_root + '/' + LBP_scoliosis_filename + '.csv', encoding = 'utf8',dtype=str)

In [5]:
print(LBP_other_all_data.keys())


Index(['EMPI', 'MGH_MRN', 'Date.prc', 'Date.dia', 'Code_Type.prc', 'Code.prc',
       'Code_Type.dia', 'Code.dia', 'Diagnosis_Name', 'Procedure_Name',
       'Gender_Legal_Sex', 'Age', 'Language', 'Race1', 'Marital_status',
       'Religion', 'Date_Of_Death', 'AgeAtPrc', 'Admit_Date', 'Discharge_Date',
       'Discharge_Disposition', 'diagnosis_list', 'Diagnosis_list_combined',
       'Diagnosis_list_no_NA', 'City', 'State', 'Zip', 'Country',
       'Insurance_1', 'Insurance_2', 'Insurance_3', 'Phone_Code',
       'Physical_Date', 'BMI', 'Height', 'Weight', 'Smoking', 'Alcohol_Use',
       'Helper'],
      dtype='object')


In [6]:
LBP_other_diagcode = LBP_other_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Diagnosis_combined','other?']]
LBP_scoliosis_diagcode = LBP_scoliosis_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Diagnosis_combined','scoliosis']]

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'LBP?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/3317639091.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/3317639091.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_diagcode['LBP?'] = str(0)


In [ ]:
LBP_and_other_scoliosis = pd.concat([LBP_other_diagcode,LBP_scoliosis_diagcode])
LBP_and_other_scoliosis['Helper'] = str(0)
LBP_and_other_scoliosis.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_other_scoliosis_copy = LBP_and_other_scoliosis
LBP_and_other_scoliosis.head(10)

In [ ]:
for i in range(1,len(LBP_and_other)):
    if LBP_and_other.loc[i,'EMPI'] == LBP_and_other.loc[i-1,'EMPI']:
        if LBP_and_other.loc[i,'Diagnosis_combined'] != LBP_and_other.loc[i-1,'Diagnosis_combined']:
            LBP_and_other.loc[i,'Diagnosis_combined'] = LBP_and_other.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_other.loc[i-1,'Diagnosis_combined'])
        if LBP_and_other.loc[i,'LBP?'] != 'LBP':
            if LBP_and_other.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_other.loc[i,'LBP?'] = 'LBP'
        if LBP_and_other.loc[i,'other?'] != 'other':
            if LBP_and_other.loc[i-1,'other?'] == 'other':
                LBP_and_other.loc[i,'other?'] = 'other'
    else:
        LBP_and_other.loc[i-1,'Helper'] = 'useful'

In [7]:
LBP_diagcode.loc[0,'Diagnosis_combined'] = LBP_diagcode.loc[0,'Code.dia']
for i in range(1,len(LBP_diagcode)):
    if (LBP_diagcode.loc[i,'EMPI'] == LBP_diagcode.loc[i-1,'EMPI']):
        if LBP_diagcode.loc[i,'Code.dia'] != LBP_diagcode.loc[i-1,'Code.dia']:
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia'] + ', ' + str(LBP_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        LBP_diagcode.loc[i,'Diagnosis_combined'] = LBP_diagcode.loc[i,'Code.dia']
        continue;

In [8]:
LBP_filtered = LBP_diagcode[LBP_diagcode['Helper'].notna()]
LBP_filtered_save = LBP_filtered

In [9]:
#CPT Code
#LBP only
incl_cpts_LBP = get_cpts_fromfile(path_inclusion_crit_LBP)
incl_cpts_text_LBP = '|'.join(incl_cpts_LBP.str.strip())

#other
incl_cpts_other = get_cpts_fromfile(path_inclusion_crit_other)
incl_cpts_text_other = '|'.join(incl_cpts_other.str.strip())

#ICD Code
#LBP only
incl_icds_LBP = get_icds_fromfile(path_inclusion_crit_LBP)
incl_icds_text_LBP = '|'.join(incl_icds_LBP.str.strip())

#other
incl_icds_other = get_icds_fromfile(path_inclusion_crit_other)
incl_icds_text_other = '|'.join(incl_icds_other.str.strip())

In [10]:
temp_df_LBP = pd.DataFrame
temp_df_LBP = LBP_filtered['Diagnosis_combined'].str.contains(incl_icds_text_LBP, regex=True)

LBP_filtered.index = range(len(LBP_filtered.index))

for i in range(len(temp_df_LBP)):
    if temp_df_LBP.iloc[i] == True:
        LBP_filtered.loc[i,'LBP?'] = 'LBP'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/484879454.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBP_filtered.loc[i,'LBP?'] = 'LBP'


In [11]:
other_diagcode = other_all_data[['EMPI','Age','Race1','BMI','Code.prc','Code.dia','Helper']]
other_diagcode['Diagnosis_combined'] = str(0)
other_diagcode['other?'] = str(0)
print(other_diagcode.keys())

other_diagcode_reference = other_diagcode

Index(['EMPI', 'Age', 'Race1', 'BMI', 'Code.prc', 'Code.dia', 'Helper',
       'Diagnosis_combined', 'other?'],
      dtype='object')


/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/1774246549.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_diagcode['Diagnosis_combined'] = str(0)
/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/1774246549.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_diagcode['other?'] = str(0)


In [12]:
other_diagcode.loc[0,'Diagnosis_combined'] = other_diagcode.loc[0,'Code.dia']
for i in range(1,len(other_diagcode)):
    if (other_diagcode.loc[i,'EMPI'] == other_diagcode.loc[i-1,'EMPI']):
        if other_diagcode.loc[i,'Code.dia'] != other_diagcode.loc[i-1,'Code.dia']:
            other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i,'Code.dia'] + ', ' + str(other_diagcode.loc[i-1,'Diagnosis_combined'])
            continue;
        else: 
            other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i-1,'Diagnosis_combined']
            continue;
    else: 
        other_diagcode.loc[i,'Diagnosis_combined'] = other_diagcode.loc[i,'Code.dia']
        continue;

In [13]:
other_filtered = other_diagcode[other_diagcode['Helper'].notna()]
other_filtered_save = other_filtered

In [14]:
temp_df_other = pd.DataFrame
temp_df_other = other_filtered['Diagnosis_combined'].str.contains(incl_icds_text_other, regex=True)

other_filtered.index = range(len(other_filtered.index))

for i in range(len(temp_df_other)):
    if temp_df_other.iloc[i] == True:
        other_filtered.loc[i,'other?'] = 'other'

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/915588701.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  other_filtered.loc[i,'other?'] = 'other'


In [15]:
LBP_and_other = pd.concat([LBP_filtered,other_filtered])

In [16]:
LBP_and_other.index = range(len(LBP_and_other.index))
LBP_and_other['Helper'] = str(0)

In [17]:
LBP_and_other.sort_values(by=['EMPI'], inplace=True, ignore_index=True)
LBP_and_other_copy = LBP_and_other
LBP_and_other.head(10)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?
0,100000152,81,White,27.02,72100,G89.29,0,G89.29,LBP,NaN
1,100000152,81,White,27.02,72100,M81.0,0,M81.0,NaN,other
2,100000243,92,Declined,24.73,72040,M54.9,0,M54.9,LBP,NaN
3,100001760,88,White,NaN,72040,G89.29,0,G89.29,LBP,NaN
4,100003506,96,Declined,20.06,72100,M54.9,0,M54.9,LBP,NaN
5,100004276,92,Other,17.51,72040,G89.29,0,G89.29,LBP,NaN
6,100004276,92,Other,21.15,72100,M54.9,0,M54.9,LBP,NaN
7,100004445,71,Black,NaN,74000,M81.0,0,M81.0,NaN,other
8,100004610,98,White,NaN,72040,M81.0,0,M81.0,NaN,other
9,100004610,98,White,NaN,72100,S30.0XXA,0,S30.0XXA,LBP,NaN


In [18]:
for i in range(1,len(LBP_and_other)):
    if LBP_and_other.loc[i,'EMPI'] == LBP_and_other.loc[i-1,'EMPI']:
        if LBP_and_other.loc[i,'Diagnosis_combined'] != LBP_and_other.loc[i-1,'Diagnosis_combined']:
            LBP_and_other.loc[i,'Diagnosis_combined'] = LBP_and_other.loc[i,'Diagnosis_combined'] + ', ' + str(LBP_and_other.loc[i-1,'Diagnosis_combined'])
        if LBP_and_other.loc[i,'LBP?'] != 'LBP':
            if LBP_and_other.loc[i-1,'LBP?'] == 'LBP':
                LBP_and_other.loc[i,'LBP?'] = 'LBP'
        if LBP_and_other.loc[i,'other?'] != 'other':
            if LBP_and_other.loc[i-1,'other?'] == 'other':
                LBP_and_other.loc[i,'other?'] = 'other'
    else:
        LBP_and_other.loc[i-1,'Helper'] = 'useful'

In [19]:
LBPnother_filtered = LBP_and_other.loc[LBP_and_other['Helper'] == 'useful']
LBPnother_filtered.reset_index(drop =True, inplace=True)
LBPnother_filtered['Both'] = str(0)

/var/folders/m6/_674l8815692m04rgcb9n31h0000gn/T/ipykernel_18708/1450135962.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  LBPnother_filtered['Both'] = str(0)


In [20]:
for i in range(0,len(LBPnother_filtered)):
    if LBPnother_filtered.loc[i,'LBP?'] == 'LBP':
        if LBPnother_filtered.loc[i,'other?'] == 'other':
            LBPnother_filtered.loc[i,'Both'] = 'Both'

In [21]:
LBPnother_filtered.head(20)

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,Both
0,100000152,81,White,27.02,72100,M81.0,useful,"M81.0, G89.29",LBP,other,Both
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP,NaN,0
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP,NaN,0
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP,NaN,0
4,100004276,92,Other,21.15,72100,M54.9,useful,"M54.9, G89.29",LBP,NaN,0
5,100004445,71,Black,NaN,74000,M81.0,useful,M81.0,NaN,other,0
6,100004610,98,White,NaN,72100,S30.0XXA,useful,"S30.0XXA, M81.0",LBP,other,Both
7,100006144,78,Black,24.12,72100,M81.0,useful,"M81.0, M43.27, M81.0, G89.29, M54.9, M43.27",LBP,other,Both
8,100006348,80,White,27,72040,M81.0,useful,"M81.0, M81.0, G89.29, M43.22",LBP,other,Both
9,100007265,88,White,27.91,72100,M81.0,useful,"M81.0, G89.29",LBP,other,Both


In [22]:
LBP_only = LBPnother_filtered.loc[LBPnother_filtered['LBP?'] == 'LBP']
LBP_only = LBP_only[LBP_only['Both'] != 'Both']
other_only = LBPnother_filtered.loc[LBPnother_filtered['other?'] == 'other']
other_only = other_only[other_only['Both'] != 'Both']
LBPother_Both_only = LBPnother_filtered.loc[LBPnother_filtered['Both'] == 'Both']

In [23]:
LBP_and_other_total = pd.concat([LBP_diagcode,other_diagcode])
LBP_and_other_total.drop_duplicates(subset=['EMPI'],inplace=True)
len(LBP_and_other_total)

5098

In [24]:
print('Number of Total Patients(LBP+other):  ' + str(len(LBP_and_other_total)-1))
print('Number of LBP only:  ' + str(len(LBP_only)))
print('Number of other only:  ' + str(len(other_only)))
print('Number of Both only:  ' + str(len(LBPother_Both_only)))

#LBP_filtered = LBP
#other_filtered = other
#LBP & other filtered = LBPnother_filtered


Number of Total Patients(LBP+other):  5097
Number of LBP only:  2841
Number of other only:  772
Number of Both only:  1479


In [25]:
LBP_patients = LBP_filtered.sort_values(by=['EMPI'], ignore_index=True)
LBP_patients.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?
0,100000152,81,White,27.02,72100,G89.29,useful,G89.29,LBP
1,100000243,92,Declined,24.73,72040,M54.9,useful,M54.9,LBP
2,100001760,88,White,NaN,72040,G89.29,useful,G89.29,LBP
3,100003506,96,Declined,20.06,72100,M54.9,useful,M54.9,LBP
4,100004276,92,Other,17.51,72040,G89.29,useful,G89.29,LBP


In [26]:
other_csv_path = os.path.join(filepath_root,'LBP_other_patients.csv')
LBPother_Both_only.to_csv(other_csv_path)

In [27]:
other_only_csv_path = os.path.join(filepath_root,'other_patients.csv')
other_filtered.to_csv(other_only_csv_path)

In [28]:
LBPother_Both_only.head()

,EMPI,Age,Race1,BMI,Code.prc,Code.dia,Helper,Diagnosis_combined,LBP?,other?,Both
0,100000152,81,White,27.02,72100,M81.0,useful,"M81.0, G89.29",LBP,other,Both
6,100004610,98,White,NaN,72100,S30.0XXA,useful,"S30.0XXA, M81.0",LBP,other,Both
7,100006144,78,Black,24.12,72100,M81.0,useful,"M81.0, M43.27, M81.0, G89.29, M54.9, M43.27",LBP,other,Both
8,100006348,80,White,27,72040,M81.0,useful,"M81.0, M81.0, G89.29, M43.22",LBP,other,Both
9,100007265,88,White,27.91,72100,M81.0,useful,"M81.0, G89.29",LBP,other,Both


Other + scoliosis
Other was filtered above --> filter scoliosis and combine

In [ ]:
scoliosis_filename = 'scoliosis_all'
scoliosis_all_data = pd.read_csv(filepath_root + '/' + scoliosis_filename + '.csv', encoding = 'utf8',dtype=str)